In [1]:
import numpy as np

# Load the parity check matrix of the classical code that is used to construct the hgp code
h = np.loadtxt('/home/normaluser/ck/linkequantum-qLDPC/quits/parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt', dtype=int)
print('Shape of classical code\'s parity check matrix: ', h.shape)

Shape of classical code's parity check matrix:  (9, 12)


In [2]:
from quits.qldpc_code import *

code = HgpCode(h, h)         # Define the HgpCode object
code.build_graph(seed=22)    # Build the Tanner graph and assign directions to its edges. 
                             # For this specific h, seed=22 gives a circuit with entangling depth 8. 
num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

print("---------------------------------------------------")

print("比特数目: ", num_data + num_zcheck + num_xcheck)

import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX

p = 1e-3           # physical error rate
num_rounds = 1    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

idle_error = p     # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
sqgate_error = p   # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
tqgate_error = p   # Float (DEPOLARIZE2 error with rate p) or tuple of 15 floats (pix, ..., pzz)
spam_error = p     # Float

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, idle_error, sqgate_error, tqgate_error, spam_error, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.

print(circuit.without_noise())

# data qubits:  225  # logical qubits:  9
# z-check qubits:  108  # x-check qubits:  108
# layers of entangling gates:  8
---------------------------------------------------
比特数目:  441
R 0 1 2 3 4 5 6 7 8 9 10 11 21 22 23 24 25 26 27 28 29 30 31 32 42 43 44 45 46 47 48 49 50 51 52 53 63 64 65 66 67 68 69 70 71 72 73 74 84 85 86 87 88 89 90 91 92 93 94 95 105 106 107 108 109 110 111 112 113 114 115 116 126 127 128 129 130 131 132 133 134 135 136 137 147 148 149 150 151 152 153 154 155 156 157 158 168 169 170 171 172 173 174 175 176 177 178 179 189 190 191 192 193 194 195 196 197 198 199 200 210 211 212 213 214 215 216 217 218 219 220 221 231 232 233 234 235 236 237 238 239 240 241 242 264 265 266 267 268 269 270 271 272 285 286 287 288 289 290 291 292 293 306 307 308 309 310 311 312 313 314 327 328 329 330 331 332 333 334 335 348 349 350 351 352 353 354 355 356 369 370 371 372 373 374 375 376 377 390 391 392 393 394 395 396 397 398 411 412 413 414 415 416 417 418 419 432 433 434 435 436

In [3]:
no_noise_circuit = circuit.without_noise()

OpenQASM不支持噪声。在2.0版本中，不支持对于经典寄存器数据的操作。（不支持Detector和OBSERVABLE_INCLUDE）

In [4]:
qasm = no_noise_circuit.to_qasm(open_qasm_version = 2, skip_dets_and_obs = True)

print(qasm.strip().replace('\n\n', '\n'))

OPENQASM 2.0;
include "qelib1.inc";
qreg q[441];
creg rec[657];
reset q[0];
reset q[1];
reset q[2];
reset q[3];
reset q[4];
reset q[5];
reset q[6];
reset q[7];
reset q[8];
reset q[9];
reset q[10];
reset q[11];
reset q[21];
reset q[22];
reset q[23];
reset q[24];
reset q[25];
reset q[26];
reset q[27];
reset q[28];
reset q[29];
reset q[30];
reset q[31];
reset q[32];
reset q[42];
reset q[43];
reset q[44];
reset q[45];
reset q[46];
reset q[47];
reset q[48];
reset q[49];
reset q[50];
reset q[51];
reset q[52];
reset q[53];
reset q[63];
reset q[64];
reset q[65];
reset q[66];
reset q[67];
reset q[68];
reset q[69];
reset q[70];
reset q[71];
reset q[72];
reset q[73];
reset q[74];
reset q[84];
reset q[85];
reset q[86];
reset q[87];
reset q[88];
reset q[89];
reset q[90];
reset q[91];
reset q[92];
reset q[93];
reset q[94];
reset q[95];
reset q[105];
reset q[106];
reset q[107];
reset q[108];
reset q[109];
reset q[110];
reset q[111];
reset q[112];
reset q[113];
reset q[114];
reset q[115];
reset q[116]

In [5]:
import re

def process_qasm_for_neutral_atoms(qasm_str, total_qubits, num_aux_qubits, num_rounds):
    """
    Process QASM string to handle auxiliary qubit renumbering for neutral atom systems
    across multiple rounds of measure-reset (MR) operations.

    Args:
        qasm_str (str):
            Full QASM string.
        total_qubits (int):
            Number of data + ancilla (Z/X checks).
        num_aux_qubits (int):
            Number of auxiliary qubits used in each MR-round.
        num_rounds (int):
            Total rounds of MR, used to compute final qreg size.

    Returns:
        str: Modified QASM string with updated auxiliary-qubit renumbering.
    """

    # --- Basic definitions ---
    lines = qasm_str.split('\n')
    MR_PATTERN = re.compile(
        r'^measure q\[(\d+)\] -> rec\[\d+\]; reset q\[\1\]; // decomposed MR$'
    )

    # --- STEP 1: Identify all auxiliary qubits involved in MR ---
    aux_qubits = []
    for line in lines:
        m = MR_PATTERN.match(line.strip())
        if m:
            aux_qubits.append(int(m.group(1)))

    aux_qubits = sorted(set(aux_qubits))
    # print(f"[DEBUG] Identified auxiliary qubits: {aux_qubits}")

    # --- STEP 2: Initial mapping (identity) ---
    #     aux_map[x] = new_index_of_qubit_x
    aux_map = {aux: aux for aux in aux_qubits}

    # --- STEP 3: Process QASM lines round-by-round ---
    processed_lines = []
    i = 0
    round_counter = 0

    while i < len(lines):
        line = lines[i].strip()

        # Case A: Start of MR group
        if MR_PATTERN.match(line):
            round_counter += 1
            mr_group = []

            # Collect this entire MR block
            while i < len(lines) and MR_PATTERN.match(lines[i].strip()):
                mr_group.append(lines[i])
                i += 1

            # Apply current aux_map to MR lines
            for idx in range(len(mr_group)):
                updated_line = mr_group[idx]
                for old, new in aux_map.items():
                    pattern = rf'(?<!\w)q\[{old}\](?!\w)'
                    updated_line = re.sub(pattern, f"q[{new}]", updated_line)
                mr_group[idx] = updated_line

            processed_lines.extend(mr_group)

            # Update mapping **for next MR round**:
            # Each round allocates a fresh block of num_aux_qubits.
            new_base = total_qubits + num_aux_qubits * (round_counter - 1)
            for k, old_aux in enumerate(aux_qubits):
                aux_map[old_aux] = new_base + k

        # Case B: Non-MR line
        else:
            updated_line = lines[i]
            for old, new in aux_map.items():
                pattern = rf'(?<!\w)q\[{old}\](?!\w)'
                updated_line = re.sub(pattern, f"q[{new}]", updated_line)

            processed_lines.append(updated_line)
            i += 1

    # --- STEP 4: Update qreg size based on formula ---
    target_total_qubits = total_qubits + num_aux_qubits * (num_rounds - 1)

    for idx, line in enumerate(processed_lines):
        if line.startswith("qreg q["):
            processed_lines[idx] = re.sub(
                r"qreg q\[\d+\]",
                f"qreg q[{target_total_qubits}]",
                line
            )
            break
    # return "\n".join(processed_lines).strip()
    # --- STEP 5: Remove `reset q[...] ; // decomposed MR` from MR lines ---
    cleaned_lines = []
    for line in processed_lines:
        cleaned_line = re.sub(
            r'reset\s+q\[\d+\];\s*//\s*decomposed MR',
            '',
            line
        ).rstrip()  # 去掉多余空格
        cleaned_lines.append(cleaned_line)

    
    return "\n".join(cleaned_lines).strip()

# Example QASM string (simplified for demonstration, based on typical output from Stim circuit)
example_qasm = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg rec[4];

// Initial operations on data and aux qubits
cx q[0], q[5];
cx q[1], q[6];

// First round of MR on aux qubits (assume q[5], q[6], q[7], q[8] are aux)
measure q[5] -> rec[0]; reset q[5]; // decomposed MR
measure q[6] -> rec[1]; reset q[6]; // decomposed MR
measure q[7] -> rec[2]; reset q[7]; // decomposed MR
measure q[8] -> rec[3]; reset q[8]; // decomposed MR

// Subsequent operations (next round, using same aux qubits initially)
cx q[0], q[5];
cx q[1], q[6];
cx q[2], q[7];
cx q[3], q[8];

h q[5];
h q[6];

// Second round of MR
measure q[5] -> rec[0]; reset q[5]; // decomposed MR
measure q[6] -> rec[1]; reset q[6]; // decomposed MR
measure q[7] -> rec[2]; reset q[7]; // decomposed MR
measure q[8] -> rec[3]; reset q[8]; // decomposed MR
"""

# Process the example QASM
processed_example = process_qasm_for_neutral_atoms(example_qasm, total_qubits=10,
                                                    num_aux_qubits=4, num_rounds=2)

print("Original QASM:")
print(example_qasm)
print("\nProcessed QASM:")
print(processed_example)

Original QASM:
OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg rec[4];

// Initial operations on data and aux qubits
cx q[0], q[5];
cx q[1], q[6];

// First round of MR on aux qubits (assume q[5], q[6], q[7], q[8] are aux)
measure q[5] -> rec[0]; reset q[5]; // decomposed MR
measure q[6] -> rec[1]; reset q[6]; // decomposed MR
measure q[7] -> rec[2]; reset q[7]; // decomposed MR
measure q[8] -> rec[3]; reset q[8]; // decomposed MR

// Subsequent operations (next round, using same aux qubits initially)
cx q[0], q[5];
cx q[1], q[6];
cx q[2], q[7];
cx q[3], q[8];

h q[5];
h q[6];

// Second round of MR
measure q[5] -> rec[0]; reset q[5]; // decomposed MR
measure q[6] -> rec[1]; reset q[6]; // decomposed MR
measure q[7] -> rec[2]; reset q[7]; // decomposed MR
measure q[8] -> rec[3]; reset q[8]; // decomposed MR


Processed QASM:
OPENQASM 2.0;
include "qelib1.inc";
qreg q[14];
creg rec[4];

// Initial operations on data and aux qubits
cx q[0], q[5];
cx q[1], q[6];

// First round of MR

In [6]:
finally_qasm = process_qasm_for_neutral_atoms(qasm, total_qubits=num_data + num_zcheck + num_xcheck,
                                              num_aux_qubits=num_zcheck+num_xcheck, num_rounds=num_rounds+1)
print("\nFinal Processed QASM:")
print(finally_qasm.strip().replace('\n\n', '\n'))


Final Processed QASM:
OPENQASM 2.0;
include "qelib1.inc";
qreg q[657];
creg rec[657];
reset q[0];
reset q[1];
reset q[2];
reset q[3];
reset q[4];
reset q[5];
reset q[6];
reset q[7];
reset q[8];
reset q[9];
reset q[10];
reset q[11];
reset q[21];
reset q[22];
reset q[23];
reset q[24];
reset q[25];
reset q[26];
reset q[27];
reset q[28];
reset q[29];
reset q[30];
reset q[31];
reset q[32];
reset q[42];
reset q[43];
reset q[44];
reset q[45];
reset q[46];
reset q[47];
reset q[48];
reset q[49];
reset q[50];
reset q[51];
reset q[52];
reset q[53];
reset q[63];
reset q[64];
reset q[65];
reset q[66];
reset q[67];
reset q[68];
reset q[69];
reset q[70];
reset q[71];
reset q[72];
reset q[73];
reset q[74];
reset q[84];
reset q[85];
reset q[86];
reset q[87];
reset q[88];
reset q[89];
reset q[90];
reset q[91];
reset q[92];
reset q[93];
reset q[94];
reset q[95];
reset q[105];
reset q[106];
reset q[107];
reset q[108];
reset q[109];
reset q[110];
reset q[111];
reset q[112];
reset q[113];
reset q[114];
res

转换为qasm文件

In [7]:
import os
target_directory = "/home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_qasm_circuit"
file_name = f"hgp_code_n_441_k_9_r_{num_rounds+1}.qasm"
# 2. 构造完整的文件路径
full_file_path = os.path.join(target_directory, file_name)

# 3. 确保目标文件夹存在
try:
    # 递归创建目录。如果目录已存在，不会抛出错误 (exist_ok=True)。
    os.makedirs(target_directory, exist_ok=True)
    print("目标文件夹已准备就绪。")
except OSError as e:
    print(f"创建目标文件夹时发生错误: {e}")
    # 如果文件夹创建失败（例如，权限不足），则停止执行
    exit()


# 4. 将 QASM 字符串写入文件
try:
    # 'w' 模式: 写入 (如果文件不存在则创建，如果存在则覆盖)
    with open(full_file_path, 'w', encoding='utf-8') as f:
        f.write(finally_qasm)
    
    print(f"\n✅ 成功将 QASM 代码保存到: {full_file_path}")

except IOError as e:
    print(f"\n❌ 保存文件时发生 I/O 错误: {e}")
except Exception as e:
    print(f"\n❌ 发生意外错误: {e}")

目标文件夹已准备就绪。

✅ 成功将 QASM 代码保存到: /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_qasm_circuit/hgp_code_n_441_k_9_r_2.qasm
